In [15]:
import math
import os
import openpyxl
import platform
import numpy as np
import pandas as pd

In [2]:
if platform.platform()[:3].lower() == 'mac':
    mac_m1 = True
elif platform.platform()[:3].lower() == 'win':
    mac_m1 = False
    
if mac_m1:
    path = '/Users/rainyseason/winston/Workspace/python/Pycharm Project/sinewave_analyze/Evaluation/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv'
else:
    path = 'D:/winston/OneDrive - (주)필드큐어/data_analyze/'
    path_csv = path + 'tek_csv/'
    path_excel = path + 'tek_excel/'
    path_summary = path + 'summary/'
    path_information = path + 'test information/'
    path_kmon = path + 'kmon_csv/'
    
evaluation_control_file = 'eval_control.xlsx'

In [13]:
filename = 'summary.xlsx'
sheets = ['with kmon Curr', 'with kmon Volt']
gather_cols = [['Irms[mA]'], ['Vpeak[V]']]

In [4]:
idx = 1
sheet = 'with kmon Volt'

In [6]:
df_summary = pd.read_excel(path_excel + filename, sheet_name=sheet)

In [7]:
remain_columns = ['Board', 'ohm', 'PWM', 'Volt[V]', 'Curr[mA]']
# remain_columns = ['Board', 'ohm']
added_item = []

In [8]:
delete_columns = list(df_summary.columns)

In [9]:
if 'curr' in sheet.lower():
    # remain_columns.insert(2, 'Curr[mA]')
    added_item.append('Curr[mA]')
    sheet_name = sheet + ' ' + 'All Ch'
if 'volt' in sheet.lower():
    # remain_columns.insert(2, 'Volt[V]')
    added_item.append('Volt[V]')
    sheet_name = sheet + ' ' + 'All Ch'
if 'pwm' in sheet.lower():
    # remain_columns.insert(2, 'PWM')
    added_item.append('PWM')
    sheet_name = sheet + ' ' + 'All Ch'

In [10]:
df = pd.DataFrame(columns=remain_columns)

In [11]:
blank_value = []
for i in range(len(df.columns)):
    blank_value.append('')

max_length_ch = df_summary['Ch'].value_counts()
max_ch = max_length_ch.index[0]

max_length_ch = df_summary['Ch'].value_counts()
max_length_ch = max_length_ch.index[0]

start = 0
end = 0
before = ''
for i in range(len(df_summary)):
    if df_summary.at[i, 'Ch'] == max_ch and before == '':
        start = i
        before = df_summary.at[i, 'Ch']
    elif before != df_summary.at[i, 'Ch']:
        end = i - 1
        break
    elif before == df_summary.at[i, 'Ch'] and i == len(df_summary) - 1:
        end = i

for i in range(start, end + 1):
    df.loc[i - start] = blank_value

for index, column in enumerate(remain_columns):
    for i in range(start, end + 1):
        a = df_summary.at[i, column]
        df.at[i - start, column] = df_summary.at[i, column]

In [16]:
add_columns = []
channels = df_summary['Ch'].unique()
channels.sort()
for col in gather_cols[idx]:
    for ch in channels:
        # dict.setdefault(col + ch, [])
        add_columns.append(col + ch)
# channels = list(dict.keys())
channels = add_columns

for col in channels:
    df[col] = np.nan

In [17]:
remain_columns = remain_columns[2:]

In [23]:
row = 0
inner_row = 0
while True:
    for col in channels:
        if df_summary.at[row, 'Ch'] in col:
            column = col.split(df_summary.at[row, 'Ch'])[0]
            while True:
                same_flag = True
                for set_value in remain_columns:
                    if df.at[inner_row, set_value] != df_summary.at[row, set_value]:
                        same_flag = False
                inner_row += 1
                if same_flag:
                    df.at[inner_row - 1, col] = df_summary.at[row, column]
                    break
    if inner_row == end + 1:
        inner_row = 0
    row += 1
    if row == len(df_summary):
        break

In [24]:
df

,Board,ohm,PWM,Volt[V],Curr[mA],Vpeak[V]Ch1,Vpeak[V]Ch2,Vpeak[V]Ch3,Vpeak[V]Ch4
0,RFAMP_03,500,-,0,-,NaN,NaN,NaN,NaN
1,RFAMP_03,500,-,1,-,NaN,NaN,NaN,NaN
2,RFAMP_03,500,-,2,-,2.952396,3.056984,2.847000,3.301275
3,RFAMP_03,500,-,3,-,NaN,3.863062,3.810181,3.551295
4,RFAMP_03,500,-,4,-,5.600466,4.354978,5.823217,5.076421
...,...,...,...,...,...,...,...,...,...
126,RFAMP_03,500,-,126,-,121.378618,117.666810,118.731287,115.751034
127,RFAMP_03,500,-,127,-,NaN,117.921908,118.894735,115.908283
128,RFAMP_03,500,-,128,-,123.180843,118.225844,119.219596,116.250352
129,RFAMP_03,500,-,129,-,NaN,118.540556,119.422453,116.477159


In [22]:
df_summary['Ch'].value_counts()

Ch4    131
Ch3    131
Ch2    131
Ch1     66
Name: Ch, dtype: int64